In [1]:
%load_ext autoreload
%autoreload 2

# Search Router

In [1]:
from src.agent.nodes.search_router import build_search_config
# ============================================================
# 테스트 코드 (파일을 직접 실행할 때만 동작)
# ============================================================

# 다양한 유형의 질문으로 테스트
test_querys = [
    "RAG가 뭐야?",                              # ML 개념 질문 (basic)
    "딥러닝 모델 최적화 방법",                    # ML 고급 질문 (advanced)
    "iris 데이터셋 불러오는 코드",               # Python 코드 질문 (basic)
    "Python list comprehension 문법",          # Python 문법 질문 (syntax)
    "RAG 구현할 때 pandas DataFrame 활용법"     # 복합 질문 (ML + Python)
]

# 테스트 결과 헤더
print("=" * 80)
print("Search Router - 완전 LLM 기반 테스트")
print("=" * 80)

# 각 질문에 대해 검색 설정 생성 및 출력
for query in test_querys:
    print(f"\n📌 질문: {query}")
    print("-" * 80)
    
    
    config = build_search_config(query)
    
    
    print(f"✅ 검색 대상: {config['sources']}")        # 어디서 검색할지
    print(f"📊 검색 개수: {config['top_k']}개")        # 몇 개 가져올지
    print(f"🔍 검색 방법: {config['search_method']}")  # 어떤 방법으로
    
    # 디버깅용 분석 정보 출력
    analysis = config['_analysis']
    print(f"\n💡 분석 정보:")
    print(f"   - 질문 유형: {analysis['query_type']}")       # concept/code/syntax
    print(f"   - 주요 키워드: {', '.join(analysis['topic_keywords'])}")  # 추출된 키워드
    print(f"   - 난이도: {analysis['complexity']}")             # basic/intermediate/advanced
    print("=" * 80)



Search Router - 완전 LLM 기반 테스트

📌 질문: RAG가 뭐야?
--------------------------------------------------------------------------------
✅ 검색 대상: ['lecture']
📊 검색 개수: 3개
🔍 검색 방법: similarity

💡 분석 정보:
   - 질문 유형: concept
   - 주요 키워드: rag
   - 난이도: basic

📌 질문: 딥러닝 모델 최적화 방법
--------------------------------------------------------------------------------
✅ 검색 대상: ['lecture']
📊 검색 개수: 7개
🔍 검색 방법: mmr

💡 분석 정보:
   - 질문 유형: concept
   - 주요 키워드: 딥러닝, 모델, 최적화
   - 난이도: advanced

📌 질문: iris 데이터셋 불러오는 코드
--------------------------------------------------------------------------------
✅ 검색 대상: ['lecture', 'python_doc']
📊 검색 개수: 3개
🔍 검색 방법: similarity

💡 분석 정보:
   - 질문 유형: code
   - 주요 키워드: iris
   - 난이도: basic

📌 질문: Python list comprehension 문법
--------------------------------------------------------------------------------
✅ 검색 대상: ['python_doc']
📊 검색 개수: 3개
🔍 검색 방법: similarity

💡 분석 정보:
   - 질문 유형: syntax
   - 주요 키워드: python, list, comprehension
   - 난이도: basic

📌 질문: RAG 구현할 때 pandas DataFrame 활용법
---

# Search Agent

In [3]:
import time
from src.agent.nodes.search_agent import is_korean, execute_dual_query_search

# ============================================================
# 테스트 실행
# ============================================================

def run_test():
    """듀얼 쿼리 검색 테스트"""
    
    # 테스트 질문 (영어 + 한글)
    test_querys = [
        # 영어 질문
        # "Using Python as a Calculator numbers operators +, -, *, /",
        # "list comprehension concise way to create lists",
        # "try except exception handling error",
        # "open file read write with statement",
        
        # 한글 질문
        "유닛/노드/뉴런 개념 알려줘.",
        "레이어, 층에 대해서 알려줘.",
        "입력층이 뭐야?",
        "머신러닝이 뭐야?",
        "결정트리가 뭐야?",
        "경사하강법 개념 알려줘",
        "결정트리와 랜덤포레스트의 차이점이 뭐야?",
        "xgboost 모델에 대해 설명해줘",
        # "지도학습이 뭐야?",
        "비지도 학습이 뭐야?",
        # "모델 불러오는 코드 예제 알려줘."
        # "리스트 컴프리헨션이란",
        # "파이썬 예외처리 방법",
        # "딕셔너리 사용법",
        # "파일 읽고 쓰는 방법",
    ]
    
    # executor = SearchExecutor()
    
    print("=" * 70)
    print("🔍 듀얼 쿼리 검색 시스템 테스트")
    print("   한글 질문 → 한글 + 영어 동시 검색")
    print("   영어 질문 → 영어만 검색")
    print("=" * 70)
    
    for i, query in enumerate(test_querys, 1):
        print(f"\n{'='*70}")
        print(f"📌 [{i}/{len(test_querys)}] 질문: {query}")
        print("-" * 70)
        
        start = time.time()
        
        try:
            # 듀얼 쿼리 검색 실행
            results, query_info = execute_dual_query_search(query)
            elapsed = time.time() - start
            
            # 결과 출력
            print(f"⏱️  검색 시간: {elapsed:.2f}초")
            print(f"🔤 원본 쿼리: {query_info['original']}")
            if query_info['translated']:
                print(f"🔄 번역 쿼리: {query_info['translated']}")
            
            print(f"\n📊 검색 결과: {len(results)}개")
            print("-" * 50)
            
            # 상위 5개 미리보기
            is_original_korean = is_korean(query_info['original'])
            
            for j, r in enumerate(results[:5], 1):
                source = r['metadata'].get('source', 'unknown')
                score = r['score']
                query_type = r.get('query_type', '?')
                
                # 쿼리 타입에 따른 이모지
                if query_type == 'original':
                    emoji = "🇰🇷" if is_original_korean else "🇺🇸"
                else:  # translated
                    emoji = "🇺🇸"
                
                preview = r['content'][:100].replace('\n', ' ')
                
                print(f"[{j}] {emoji} 유사도: {score:.4f} | 소스: {source}")
                print(f"    {preview}...")
                
        except Exception as e:
            print(f"❌ 검색 실패: {e}")
            import traceback
            traceback.print_exc()
    
    print("\n" + "=" * 70)
    print("✅ 테스트 완료")
    print("=" * 70)


if __name__ == "__main__":
    run_test()


🔍 듀얼 쿼리 검색 시스템 테스트
   한글 질문 → 한글 + 영어 동시 검색
   영어 질문 → 영어만 검색

📌 [1/9] 질문: 유닛/노드/뉴런 개념 알려줘.
----------------------------------------------------------------------
⏱️  검색 시간: 1.68초
🔤 원본 쿼리: 유닛/노드/뉴런 개념 알려줘.

📊 검색 결과: 3개
--------------------------------------------------
[1] 🇰🇷 유사도: 0.3903 | 소스: lecture
    [강의: 09_결정트리와 랜덤포레스트]  ``` LSTAT <= 8.13 # 노드를 분기하기 위한 질문 (오차가 가장 적게 나뉘도록 하는 질문) =============아래: 현재...
[2] 🇰🇷 유사도: 0.3869 | 소스: lecture
    [강의: 09_결정트리와 랜덤포레스트]  Decision Tree 알고리즘 개요   - 모델이 정답을 잘 예측 할 수 있는 질문을 던지며 대상을 좁혀가는 방식으로, '스무고개'와 ...
[3] 🇰🇷 유사도: 0.3664 | 소스: lecture
    [강의: 01_머신러닝개요]  데이터 셋 구성 Feature - 추론하기 위한 근거가 되는 값들을 표현하는 용어. - 예측 하거나 분류해야 하는 데이터의 특성, 속성 값을 말한다....

📌 [2/9] 질문: 레이어, 층에 대해서 알려줘.
----------------------------------------------------------------------
⏱️  검색 시간: 2.46초
🔤 원본 쿼리: 레이어, 층에 대해서 알려줘.

📊 검색 결과: 3개
--------------------------------------------------
[1] 🇰🇷 유사도: 0.3017 | 소스: lecture
    [강의: 13_선형모델_로지스틱회귀]  ```python # LR - weight와 bias 를 조회 pipel